# Background

Twitter is a micro-blogging social media platform with 217.5 million daily active users globally. With 500 million new tweets (posts) daily, the topics of these tweets varies widely – k-pop, politics, financial news… you name it! Individuals use it for news, entertainment, and discussions, while corporations use them to as a marketing tool to reach out to a wide audience. Given the freedom Twitter accords to its user, Twitter can provide a conducive environment for productive discourse, but this freedom can also be abused, manifesting in the forms of racism and sexism.

# Problem Statement

With Twitter’s significant income stream coming from advertisers, it is imperative that Twitter keeps a substantial user base. On the other hand, Twitter should maintain a safe space for users and provide some level of checks for the tweets the users put out into the public space, and the first step would be to identify tweets that espouse racist or sexist ideologies, and then Twitter can direct the users to appropriate sources of information where users can learn more about the community that they offend or their subconscious biases so they will be more aware of their racist/sexist tendencies. Thus, to balance, Twitter has to be accurate in filtering inappropriate tweets from innocuous ones, and the kind of inappropriateness of flagged tweets (tag - racist or sexist).

F1-scores will be the primary metric as it looks at both precision and recall, each looking at false positives (FPs) and false negatives (FNs) respectively, and is a popular metric for imbalanced data as is the case with the dataset used.

For the purpose of explanation, racist tweets are used as the ‘positive’ case.

In this context, FPs are the cases where the model erroneously flags out tweets as racist when the tweet is actually innocuous/sexist. FNs are cases where the model erroneously flags out tweets as innocuous/sexist but the tweets are actually racist.

Thus, higher F1-scores are preferred.

# Importing Libraries

In [117]:
# Standard libraries
import numpy as np
import pandas as pd

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For NLP data cleaning and preprocessing
import re, string, nltk, itertools
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import demoji

# Pickle to save model
import pickle

# For NLP Machine Learning processes
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Pipeline
from imblearn.pipeline import Pipeline

# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance

# Support Vector Machine
from sklearn.svm import SVC

# PyTorch LSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Tokenization for LSTM
from collections import Counter
from gensim.models import Word2Vec

# Transformers library for BERT
import transformers
from transformers import BertModel
from transformers import BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

# Evaluation Metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve, RocCurveDisplay
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay, classification_report, plot_confusion_matrix

In [32]:
# Setting seed for reproducibility
import random

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

In [33]:
# Changing display settings
pd.set_option('display.max_row', 100)
pd.set_option('display.max_colwidth', None)

# Importing Test Dataset

In [34]:
test = pd.read_csv('../Capstone/data/cyberbullying_tweets.csv')

In [35]:
test.columns

Index(['tweet_text', 'cyberbullying_type'], dtype='object')

In [36]:
test.shape

(47692, 2)

In [37]:
test.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was crapilicious! #mkr",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red velvet cupcakes?,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, but not too concerned about another angry dude on twitter.",not_cyberbullying
4,"@RudhoeEnglish This is an ISIS account pretending to be a Kurdish account. Like Islam, it is all lies.",not_cyberbullying


# Preprocessing Test dataset

### Aligning tags with train dataset

In [38]:
test = test.loc[(test['cyberbullying_type'] == 'not_cyberbullying') |
                (test['cyberbullying_type'] == 'ethnicity') |
                (test['cyberbullying_type'] == 'religion') |
                (test['cyberbullying_type'] == 'gender')]

In [39]:
# Checking results of filtering
test.cyberbullying_type.value_counts()

religion             7998
gender               7973
ethnicity            7961
not_cyberbullying    7945
Name: cyberbullying_type, dtype: int64

In [40]:
# Remapping flag categories - ethnicity and religion as racism (2), and gender as sexism (1)
remap = {'ethnicity': 2, 'religion': 2, 'gender': 1, 'not_cyberbullying': 0}
test = test.replace({'cyberbullying_type': remap})

In [41]:
# Checking results of remapping
test.cyberbullying_type.value_counts()

2    15959
1     7973
0     7945
Name: cyberbullying_type, dtype: int64

In [50]:
test.cyberbullying_type.value_counts(normalize = True)

2    0.500643
1    0.250118
0    0.249239
Name: cyberbullying_type, dtype: float64

Due to time and memory constraints, we will only be taking a subset of this test dataset.

In [45]:
X, y = test['tweet_text'], test['cyberbullying_type']

In [47]:
# This will ensure that the there will be half of each tag will be subsetted
X_1, X_2, y_1, y_2 = train_test_split(X, y, 
                                    train_size=0.5, 
                                    random_state= seed_value,
                                    stratify=y)

In [108]:
y_1.shape

(15938,)

In [109]:
y_1.value_counts()

2    7979
1    3986
0    3973
Name: cyberbullying_type, dtype: int64

In [110]:
y_1.value_counts(normalize = True)

2    0.500627
1    0.250094
0    0.249278
Name: cyberbullying_type, dtype: float64

The proportion of y_1 has preserved the proportion of cyberbullying_type from before the train/test split.

In [67]:
test_1 = pd.DataFrame([X_1,y_1]).T

In [68]:
test_1.head()

,tweet_text,cyberbullying_type
8876,"Olly Alexander avoids going online due to homophobic ‘rape jokes’: The Years &amp; Years frontman says... http://tinyurl.com/hmfmvoq #gay, #lgbt",1
17129,@ummahwitness @Marwan_Tunsi_ @alwallawalbara My suggestion would be that the Kuffar bomb the Kabba.,2
42753,"bitches be mad though lmao RT @_TommyPickless: Get this cum guzzling SLORE off twitter! RT @tayyoung_: FUCK OBAMA, dumb ass nigger”",2
39922,@BarackObama WOW U DUMB NIGGER&lt; FUCK U AND YO HEALTH CARE NIGGA,2
1049,Yay Instant Restaurants are over!!! #mkr,0


### Creating a function for text cleaning (lemmatizing)

In [57]:
# Instantiating stopwords
Stopwords = set(stopwords.words('english'))

In [58]:
# Updating Stopwords to align with cleaning for train dataset
Stopwords.update(['rt','amp'])

In [59]:
def clean_text_lemmatize(text):
    # Removing emojis
    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item,'')
        
    # Removing mentions and URLs
    pattern = re.compile(r"(@[A-Za-z0-9]+|_[A-Za-z0-9]+|https?://\S+|www\.\S+|\S+\.[a-z]+|)")
    text = pattern.sub('', text)
    text = " ".join(text.split())
    
    # Making text lowercase
    text = text.lower()
    
    # Decontracting constracted words
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    
    # Removing punctuations
    remove_punc = re.compile(r"[%s]" % re.escape(string.punctuation))
    text = remove_punc.sub('', text)
    
    # Lemmatizing
    # To retrieve the appropriate part-of-speech (POS) tagging for each word in a sentence/tweet for the usage of WordNetLemmatizer
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in str(text).split()]
    text = ' '.join(text)
    
    # Removing back-to-back spaces
    text = re.sub("\s\s+" , " ", text)
    
    # Removing stopwords
    text = " ".join([word for word in str(text).split() if word not in Stopwords])
    
    return text

### Creating Function to create character n-grams

In [60]:
def creating_char_n_gram(text, n):
    result = []
    text = str(text).split()
    for word in text:
        result.append([word[i: i + n] for i in range(len(word) - n + 1)])
    result = list(itertools.chain.from_iterable(result))
    return result

In [61]:
# Results of previous function is a list of lists containing character n-grams, need to unwrap the inner lists
def unwrapping_lists_of_char_n_grams(text):
    for i in range(len(text)):
        return ' '.join(text)

### Cleaning and Lemmatizing

In [70]:
# Cleaning and lemmatizing
test_1['tweet_text_lemm'] = test_1['tweet_text'].apply(lambda text: clean_text_lemmatize(text))

### Creating character 4gram column

In [71]:
# Creating a column of character 4gram
test_1['Text_lemm_char_4_gram'] = test_1['tweet_text_lemm'].apply(lambda text: creating_char_n_gram(text,4))
test_1['Text_lemm_char_4_gram'] = test_1['Text_lemm_char_4_gram'].apply(lambda text: unwrapping_lists_of_char_n_grams(text))

In [83]:
test_1.head()

,tweet_text,cyberbullying_type,tweet_text_lemm,Text_lemm_char_4_gram,text_length
8876,"Olly Alexander avoids going online due to homophobic ‘rape jokes’: The Years &amp; Years frontman says... http://tinyurl.com/hmfmvoq #gay, #lgbt",1,olly alexander avoids go online due homophobic ‘rape jokes’ year year frontman say gay lgbt,olly alex lexa exan xand ande nder avoi void oids onli nlin line homo omop moph opho phob hobi obic ‘rap rape joke okes kes’ year year fron ront ontm ntma tman lgbt,164.0
17129,@ummahwitness @Marwan_Tunsi_ @alwallawalbara My suggestion would be that the Kuffar bomb the Kabba.,2,suggestion would kuffar bomb kabba,sugg ugge gges gest esti stio tion woul ould kuff uffa ffar bomb kabb abba,74.0
42753,"bitches be mad though lmao RT @_TommyPickless: Get this cum guzzling SLORE off twitter! RT @tayyoung_: FUCK OBAMA, dumb ass nigger”",2,bitch mad though lmao get cum guzzle slore twitter fuck obama dumb nigger”,bitc itch thou houg ough lmao guzz uzzl zzle slor lore twit witt itte tter fuck obam bama dumb nigg igge gger ger”,114.0
39922,@BarackObama WOW U DUMB NIGGER&lt; FUCK U AND YO HEALTH CARE NIGGA,2,wow u dumb niggerlt fuck u yo health care nigga,dumb nigg igge gger gerl erlt fuck heal ealt alth care nigg igga,64.0
1049,Yay Instant Restaurants are over!!! #mkr,0,yay instant restaurant mkr,inst nsta stan tant rest esta stau taur aura uran rant,54.0


In [88]:
test_1.Text_lemm_char_4_gram.isna().sum()

164

Since tweet_text_lemm texts that do not have any remaining words or that the remaining words have less than 4 characters will return a None (which does not count as NaN, dropna() will not work. Thus, there is a preliminary step to replace None with np.nan first before using dropna().

In [89]:
# Dropping rows with None values under Text_lemm_char_4_gram column
test_1 = test_1.replace(to_replace='None', value=np.nan).dropna()

In [90]:
test_1.Text_lemm_char_4_gram.isna().sum()

0

In [91]:
test_1.shape

(15774, 5)

In [149]:
test_1_0s_and_1s = test_1.loc[(test_1['cyberbullying_type'] == 0) | (test_1['cyberbullying_type'] == 1)]

In [157]:
test_1_0s_and_1s.cyberbullying_type.value_counts()

1    3956
0    3841
Name: cyberbullying_type, dtype: int64

In [150]:
X1 = test_1_0s_and_1s['Text_lemm_char_4_gram'].values
y1 = test_1_0s_and_1s['cyberbullying_type'].values

In [153]:
y1 = y1.astype('int')

In [120]:
X = test_1['Text_lemm_char_4_gram'].values
y = test_1['cyberbullying_type'].values

In [145]:
y = y.astype('int')

# Loading Models

In [78]:
# Multinomial Naive Bayes
multi_nb_model = pickle.load(open("../Capstone/multi_nb.pkl", "rb"))

# Random Forest
rf_model = pickle.load(open("../Capstone/rf_model.pkl", "rb"))

# SVM
svm_model = pickle.load(open("../Capstone/svm_model.pkl", "rb"))

# Creating class for BERT model

In [80]:
%%time
class Bert_Classifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(Bert_Classifier, self).__init__()
        # Specify hidden size of BERT, hidden size of the classifier, and number of labels
        n_input = 768
        n_hidden = 50
        # 3 n_output because there are 3 categories of tweets ('none': 0, 'sexism': 1, 'racism': 2)
        n_output = 3
        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Add dense layers to perform the classification
        self.classifier = nn.Sequential(
            nn.Linear(n_input,  n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_output)
        )
        # Add possibility to freeze the BERT model
        # to avoid fine tuning BERT params (usually leads to worse results)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        # Feed input data to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: total: 0 ns
Wall time: 0 ns


In [81]:
bert10_model = pickle.load(open("../Capstone/bert_classifier_10epochs.pkl", "rb"))

# Prediction

### Multinomial Naive Bayes

In [126]:
# Predicting tags on test set
multi_nb_pred = multi_nb_model.predict(X)

In [160]:
confusion_matrix(y, multi_nb_pred)

array([[3330,  305,  206],
       [2093, 1763,  100],
       [4893,  611, 2473]], dtype=int64)

In [148]:
multilabel_confusion_matrix(y, multi_nb_pred)

array([[[ 4947,  6986],
        [  511,  3330]],

       [[10902,   916],
        [ 2193,  1763]],

       [[ 7491,   306],
        [ 5504,  2473]]], dtype=int64)

In [161]:
multi_nb_f1 = f1_score(y, multi_nb_pred, average = 'weighted')

In [162]:
multi_nb_f1

0.4803717499271705

In [163]:
multi_nb_precision = precision_score(y, multi_nb_pred, average = 'weighted')

In [165]:
multi_nb_precision

0.6936657211406784

In [164]:
multi_nb_recall = recall_score(y, multi_nb_pred, average = 'weighted')

In [166]:
multi_nb_recall

0.4796500570559148

As observed from this low F1-score, it can be observed that this Multinomial Naive Bayes model is overtrained to the train dataset and is not generalized well for the test dataset.

Since not identifying offensive tweets (False Negative) is more harmful than erroneously flagging innocuous tweets as offensive (False Positive), we can instead look at precision, i.e. proportion of positive cases that are correctly identified (innocuous tweets identified to be innocuous, racist tweets identified to be racist, and sexist tweets identified to be sexist).